In [14]:
import logging, urllib3

# Suppress all urllib3 warnings and errors
logging.getLogger("urllib3").setLevel(logging.CRITICAL)

# Also, disable specific SSL warnings if using insecure connections
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [15]:
from dotenv import load_dotenv

env_loaded = load_dotenv('../../.env-local')
env_loaded

True

In [16]:
from datadocai.models import CurrentTable

TRINO_CATALOG = 'postgres'
TRINO_SCHEMA = 'public'
TRINO_TABLE = 'house_pricing'

# construct the table you want to analyse
ct = CurrentTable(trino_catalog=TRINO_CATALOG,
                  trino_schema=TRINO_SCHEMA,
                  trino_table=TRINO_TABLE)

In [17]:
import os
from datadocai.database import DatabaseClient

# connect to trino
dc = DatabaseClient(host=os.getenv('TRINO_HOST'),
                    port=os.getenv('TRINO_PORT'),
                    user=os.getenv('TRINO_USER'),
                    password=os.getenv('TRINO_PASSWORD'))

LOCAL connection


## Show the table informations

In [18]:
cursor = dc.conn.cursor()
cursor.execute(
    f"SHOW CREATE TABLE {ct.trino_catalog}.{ct.trino_schema}.{ct.trino_table}")
data = cursor.fetchall()
description = cursor.description
cursor.close()

output = "\n\n"
# Add column name

for line in data:
    output += ", ".join([str(c) for c in line])
    output += "\n"

print(output)



CREATE TABLE postgres.public.house_pricing (
   id integer NOT NULL,
   address varchar(255),
   city varchar(100),
   state varchar(100),
   zip_code varchar(20),
   square_feet integer,
   bedrooms integer,
   bathrooms decimal(2, 1),
   listing_price decimal(12, 2),
   sale_price decimal(12, 2),
   listing_date date,
   sale_date date,
   status varchar(50)
)



In [19]:
from langchain_openai.chat_models import AzureChatOpenAI
from httpx import Client

llm = AzureChatOpenAI(
    deployment_name="gpt-4o-mini",
    openai_api_version="2024-05-01-preview",
    streaming=True,
    verbose=True,
    http_client=Client(verify=False)
)

In [20]:
from datadocai.metadata import TableMetadataManager
from datadocai.metadata.exporter.trino import MetadataTrinoExporter

metadata_exporter = MetadataTrinoExporter(current_table=ct, database_client=dc)

tmm = TableMetadataManager(current_table=ct, database_client=dc, llm=llm, metadata_exporter=metadata_exporter)

In [21]:
# launch the process
tmm.process()


        COMMENT ON TABLE postgres.public.house_pricing
        IS 'The house_pricing table contains detailed information about residential properties, including their pricing, characteristics, and status during the listing and sale process. This table is essential for analyzing housing market trends, pricing strategies, and property features.'
         The house_pricing table contains detailed information about residential properties, including their pricing, characteristics, and status during the listing and sale process. This table is essential for analyzing housing market trends, pricing strategies, and property features.
postgres.public.house_pricing Set Documentation for column id
postgres.public.house_pricing Set Documentation for column address
postgres.public.house_pricing Set Documentation for column city
postgres.public.house_pricing Set Documentation for column state
postgres.public.house_pricing Set Documentation for column zip_code
postgres.public.house_pricing Set Docume

({'input': 'Create a documentation for the table: house_pricing',
  'chat_history': [],
  'agent_outcome': DocumentationTable(description='The house_pricing table contains detailed information about residential properties, including their pricing, characteristics, and status during the listing and sale process. This table is essential for analyzing housing market trends, pricing strategies, and property features.', columns={'id': DocumentationColumn(description='Unique identifier for each property listing.'), 'address': DocumentationColumn(description='Street address of the property.'), 'city': DocumentationColumn(description='City where the property is located.'), 'state': DocumentationColumn(description='State where the property is located.'), 'zip_code': DocumentationColumn(description="Postal code for the property's location."), 'square_feet': DocumentationColumn(description='Total area of the property in square feet.'), 'bedrooms': DocumentationColumn(description='Number of bedroo

## Show the comment in the database

In [22]:
cursor = dc.conn.cursor()
cursor.execute(
    f"SHOW CREATE TABLE {ct.trino_catalog}.{ct.trino_schema}.{ct.trino_table}")
data = cursor.fetchall()
description = cursor.description
cursor.close()

output = "\n\n"
# Add column name

for line in data:
    output += ", ".join([str(c) for c in line])
    output += "\n"

print(output)



CREATE TABLE postgres.public.house_pricing (
   id integer NOT NULL COMMENT 'Unique identifier for each property listing.',
   address varchar(255) COMMENT 'Street address of the property.',
   city varchar(100) COMMENT 'City where the property is located.',
   state varchar(100) COMMENT 'State where the property is located.',
   zip_code varchar(20) COMMENT 'Postal code for the property''s location.',
   square_feet integer COMMENT 'Total area of the property in square feet.',
   bedrooms integer COMMENT 'Number of bedrooms in the property.',
   bathrooms decimal(2, 1) COMMENT 'Number of bathrooms in the property, formatted to one decimal place.',
   listing_price decimal(12, 2) COMMENT 'Initial asking price for the property when listed.',
   sale_price decimal(12, 2) COMMENT 'Final sale price of the property upon sale completion.',
   listing_date date COMMENT 'Date when the property was listed for sale.',
   sale_date date COMMENT 'Date when the property was sold. If not sold, thi